In [ ]:
# import dependencies
import json
import openrouteservice
import os
from oauth2client.service_account import ServiceAccountCredentials
import gspread


In [ ]:
# manually set lat/lng here
locations = {
    "location1": (37.7749, -122.4194),  # San Francisco
    "location2": (34.0522, -118.2437),  # Los Angeles
    "location3": (40.7128, -74.0060),   # New York City
    "location4": (41.8781, -87.6298),   # Chicago
}

In [ ]:
# authenticate and connect to google sheets
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]
creds = ServiceAccountCredentials.from_json_keyfile_name("api_keys.json", scope)
client = gspread.authorize(creds)
spreadsheet = client.open_by_key("12L4EkdRqaQ_e42fGHWaTmgCeqQrNgjTfoeAEc5AB6tw")

In [ ]:
# load route data
route_sheet = spreadsheet.worksheet("Routes")
route_data = pd.DataFrame(route_sheet.get_all_records())

In [13]:
# connect to OpenRouteService API for automobile routes
with open("api_keys.json") as f:
    api_keys = json.load(f)
ors_client = openrouteservice.Client(key=api_keys["openrouteservice"])

In [ ]:
# function to return a geojson route between two coordinates 
def fetch_route(start_coords, end_coords, transport_mode="driving-car"):
    try:
        # Fetch the route from OpenRouteService
        route = ors_client.directions(
            coordinates=[start_coords, end_coords],
            profile=transport_mode,
            format="geojson"
        )
        return route
    except openrouteservice.exceptions.ApiError as e:
        print(f"Failed to fetch route: {e}")
        return None
    

In [ ]:
# geocode coordinates a location
def geocode_location(location):
    try:
        geocode_response = ors_client.pelias_search(text=location)
        if geocode_response["features"]:
            coords = geocode_response["features"][0]["geometry"]["coordinates"]
            return coords[0], coords[1]  # Return (lon, lat)
    except Exception as e:
        print(f"Failed to geocode location {location}: {e}")
    return None, None

In [ ]:
# process GeoJSON routes and save to file
for index, row in route_data.iterrows():
    # skip extant filenames
    if pd.notna(row["filename"]) and row["filename"].strip() != "":
        # print(f"Skipping route {row['route_id']} as it already has a filename: {row['filename']}")
        continue

    # check if start_location and end_location exist in the manually set locations, default to geocode
    start_coords = locations.get(row["start_location"], geocode_location(row["start_location"]))
    end_coords = locations.get(row["end_location"], geocode_location(row["end_location"]))

    # if geocoding was successful, fetch the route
    if None not in start_coords and None not in end_coords:
        route = fetch_route(start_coords, end_coords, row["transport_mode"])

        # if route, save to GeoJSON file, update route_data df
        if route:
            geojson_filename = f"{row['route_id']}.geojson"
            geojson_path = Path("../docs/resources/geojson/") / geojson_filename

            with open(geojson_path, "w") as f:
                json.dump(route, f, indent=4)

            route_data.at[index, "filename"] = geojson_filename
            print(
                f"Generated GeoJSON for route {row['start_location']} to {row['end_location']}, and saved to {geojson_path}"
            )

Failed to fetch route: 404 ({'error': {'code': 2010, 'message': 'Could not find routable point within a radius of 350.0 meters of specified coordinate 0: -19.1817000 15.9177000.; Could not find routable point within a radius of 350.0 meters of specified coordinate 1: -22.5609000 17.0658000.'}, 'info': {'engine': {'build_date': '2025-03-14T11:07:03Z', 'graph_version': '1', 'version': '9.1.1'}, 'timestamp': 1743837194919}})
